In [1]:
import spacy
import csv
from collections import namedtuple
from spacy.matcher import PhraseMatcher

In [2]:
o = ""

for page in range(1,60):
    with open('edited_from_ocr/Complaint-p{}-normal.txt'.format(str(page)),'r') as output:
        o = "\n".join([o, output.read()])

In [3]:
o = o.replace("\n", " ").replace("Case 1:18-cv-03501 Document 1 Filed 04/20/18 ", "")\
    .replace("$$", "§§").replace(" $ ", " § ").replace("С", "C").replace(" S ", " § ")\
    .replace(" SS", " §§").replace(" 55 "," §§ ")

In [4]:
for n in range(66):
    o = o.replace(f"Page {n} of 66", f'**{n}**')
    # o = o.replace(f"Page {n} of 66", f'<page number="{n}" />')

In [5]:
# Spacy parsing needs to be done before tags are added.

o = "<complaint>" + o + "</complaint>"

In [6]:
def tag_allegations(s: str,first: int,last: int):
    s = s.replace(f" {first}. ", f"""<allegation para="{first}">""", 1)
    for n in range(first+1, last+1):
        s = s.replace(f" {n}. ", f"""</allegation><allegation para="{n}">""", 1)
    s = s.replace(f" {last+1}. ", f"""</allegation> {last+1}. """, 1) # not good, consumes the next heading.
    return s

o = tag_allegations(o, 146, 153)

In [7]:
# o

'<complaint> **1** IN THE UNITED STATES DISTRICT COURT FOR THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE,) Plaintiff, Civil Action No JURY DEMAND THE RUSSIAN FEDERATION GENERAL STAFF OF THE ARMED COMPLAINT COMPUTER FRAUD AND ABUSE ACT (18 U.S.C. 1030(a)) FORCES OF THE RUSSIAN FEDERATION ("GRU"); GRU OPERATIVE USING ) Rico (18 U.S.C. § 1962(c)) ) $1962(d) ) 25 10-22) ACT (18 U.S.C. §§ 2701-12) RICO CONSPIRACY (18 U.S.C WIRETAP ACT (18 U.S.C. STORED COMMUNICATIONS PSEUDONYM "GUCCIFER 2.0", ARAS ISKENEROVICH AGALAROV EMIN ARAZ AGALAROV JOSEPH MIFSUD WIKILEAKS JULIAN ASSANGE; DONALD J. TRUMP FOR PRESIDENT, INC., DONALD J. TRUMP, JR. PAUL J. MANAFORT, JR. ROGER J. STONE, JR.; JARED C. KUSHNER GEORGE PAPADOPOULOS RICHARD W. GATES, III; JOHN DOES 1-10, DIGITAL MILLENNIUM § 1201 et seq.) MISAPPROPRIATION OF TRADE SECRETS UNDER THE DEFEND TRADE SECRETS ACT (18 U.S.C § 1836 et seq.) WASHINGTON D.C. UNIFORM TRADE SECRETS ACT (D.C. Code Ann-55 36-401-46-410) ) Defendants. TRESPAS

In [8]:
parties = []

with open("./data/parties.csv", newline="") as infile:
    reader = csv.reader(infile)
    Party = namedtuple("Party", next(reader))  # get names from column headers
    for data in map(Party._make, reader):
        parties.append(data)
    

In [9]:
parties

[Party(role='Plaintiff', name='Democratic National Committee', short_name='DNC'),
 Party(role='Defendant', name='Russian Federation', short_name='Russia'),
 Party(role='Defendant', name='General Staff of the Armed Forces of the Russian Federation', short_name='GRU'),
 Party(role='Defendant', name='GRU operative using the pseudonym "Guccifer 2.0"', short_name='GRU Operative #1'),
 Party(role='Defendant', name='Aras Iskenerovich Agalarov', short_name='Aras Agalarov'),
 Party(role='Defendant', name='Emin Araz Agalarov', short_name='Emin Agalarov'),
 Party(role='Defendant', name='Joseph Mifsud', short_name='Mifsud'),
 Party(role='Defendant', name='WikiLeaks', short_name='Wiki Leaks'),
 Party(role='Defendant', name='Julian Assange', short_name='Assange'),
 Party(role='Defendant', name='Donald J. Trump for President, Inc.', short_name='Trump Campaign'),
 Party(role='Defendant', name='Donald J. Trump, Jr.', short_name='Trump, Jr.'),
 Party(role='Defendant', name='Paul J. Manafort, Jr.', short

In [10]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(o)

In [11]:
# This is probably overkill. I should just use string methods to highlight/tag the names.

party_matches = []

def on_match(matcher, doc, uid, matches):
    print(f'{uid} matched!')
    # party_matches.append((uid, matches, "Party"))

matcher = PhraseMatcher(nlp.vocab)
for p in parties:
    if len(p.name) < 10:
        matcher.add(p.short_name, on_match, nlp(p.short_name), nlp(p.name))
    else: matcher.add(p.short_name, on_match, nlp(p.short_name))
    
# matches = matcher(doc)

In [12]:
Entity = namedtuple("Entity", "text start_char end_char label_")

In [13]:
entities = [Entity(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents
            if e.label_ not in ("CARDINAL", "ORDINAL", "DATE", "MONEY", "TIME", "LAW", "FAC")]

In [14]:
entities

[Entity(text='THE UNITED STATES DISTRICT COURT', start_char=21, end_char=53, label_='GPE'),
 Entity(text='THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE', start_char=58, end_char=121, label_='WORK_OF_ART'),
 Entity(text='RUSSIAN', start_char=167, end_char=174, label_='NORP'),
 Entity(text='THE ARMED COMPLAINT COMPUTER', start_char=203, end_char=231, label_='ORG'),
 Entity(text='U.S.C.', start_char=256, end_char=262, label_='GPE'),
 Entity(text='FORCES OF THE RUSSIAN FEDERATION', start_char=272, end_char=304, label_='ORG'),
 Entity(text='GRU', start_char=307, end_char=310, label_='ORG'),
 Entity(text='GRU OPERATIVE', start_char=314, end_char=327, label_='ORG'),
 Entity(text='Rico', start_char=336, end_char=340, label_='PERSON'),
 Entity(text='U.S.C.', start_char=345, end_char=351, label_='GPE'),
 Entity(text='U.S.C.', start_char=394, end_char=400, label_='PERSON'),
 Entity(text='ARAS ISKENEROVICH AGALAROV', start_char=510, end_char=536, label_='ORG'),
 Entity(text='JOSE

In [15]:
for sent in list(doc.sents)[100:110]:
    print(sent.text)

(AGAINST ALL DEFENDANTS)
51 52 54 PRAYER FOR RELIE JURY DEMAND  
*
*5** Plaintiff the Democratic National Committee DNC"), brings this Complaint against The Russian Federation ("Russia"), the General Staff of the Armed Forces of the Russian Federation ("GRU"), the GRU operative using the pseudonym "Guccifer 2.0") ("GRU Operative #1"), Ars Iskenerovich Agalarov "Aras Agalarov"), Emin Araz Agalarov ("Emin Agalarov), Joseplh Mifsud ("Mifsud"), WikiLeaks, Julian Assange ("Assange"), Donald J. Trump for President, Inc the Trump Campaign"), Donald J. Trump, Jr.(Trump, Jr."), Paul J. Manafort, Jr.
"Manafort" Roger J. Stone, Jr. ("Stone"), Jared C. Kushner "Kushner"), George Papadopoulos ("Papadopoulos"), Richard W. Gates, III ("Gates"), and John Does 1-10, and alleges as follows:
NATURE OF ACTION I. INTRODUCTION .
No one is above the law.
In the run-up to the 2016 election, Russia mounted a brazen attack on American Democracy.
The opening salvo was a cyberattack on the DNC, carried out on Ame

In [16]:
doc.ents[1]

THE UNITED STATES DISTRICT COURT

In [17]:
type(doc.ents[1])

spacy.tokens.span.Span

In [18]:
# doc.ents

(1,
 THE UNITED STATES DISTRICT COURT,
 THE SOUTHERN DISTRICT OF NEW YORK DEMOCRATIC NATIONAL COMMITTEE,
 RUSSIAN,
 THE ARMED COMPLAINT COMPUTER,
 18,
 U.S.C.,
 FORCES OF THE RUSSIAN FEDERATION,
 GRU,
 GRU OPERATIVE,
 Rico,
 18,
 U.S.C.,
 1962(d,
 25,
 18,
 U.S.C.,
 2701-12,
 RICO,
 18,
 U.S.C WIRETAP ACT,
 18,
 2.0,
 ARAS ISKENEROVICH AGALAROV,
 JOSEPH MIFSUD,
 DONALD J. TRUMP,
 INC,
 DONALD J. TRUMP,
 JR,
 PAUL J. MANAFORT,
 JR,
 ROGER J. STONE,
 JR,
 JARED C. KUSHNER,
 RICHARD W. GATES,
 JOHN,
 1-10,
 DIGITAL MILLENNIUM,
 1201,
 THE DEFEND TRADE SECRETS ACT,
 18,
 U.S.C,
 1836,
 WASHINGTON,
 D.C.,
 Ann-55,
 36-401,
 TRESPASS,
 CHATTELS,
 Virginia Common Law),
 CHATTELS,
 Virginia Common Law,
 VIRGINIA COMPUTER CRIMES,
 Va.,
 Code Ann.18.2-152.5 et seq,
 2,
 DNC,
 VENUE,
 TRUMP ASSOCIATES',
 PRE-EXISTING RELATIONSHIPS,
 13,
 RUSSIA,
 RUSSIAN,
 RUSSIA,
 15,
 DNC,
 17,
 TRUMP A,
 Trump Announces His,
 Russia,
 Begins,
 18,
 B. European,
 U.S.,
 Trump Signs a Letter of Intent,
 D.,
 The

In [19]:
o = o.replace("&well","and well").replace(" & ",", ")

In [20]:
import xml.etree.ElementTree as ET
root = ET.fromstring(o)

In [24]:
root.getchildren()

[<Element 'allegation' at 0x114a61c28>,
 <Element 'allegation' at 0x114a61318>,
 <Element 'allegation' at 0x114a61f48>,
 <Element 'allegation' at 0x10f6f2048>,
 <Element 'allegation' at 0x10f6f2098>,
 <Element 'allegation' at 0x10f6f20e8>,
 <Element 'allegation' at 0x10f6f2138>]

In [28]:
for a in root.getchildren():
    print(a.text)
    print("")

Plaintiff realleges and incorporates by reference all prior paragraphs of this Complaint and paragraphs in the counts below as though set forth fully herein.

The DNC's computers are involved in interstate and foreign commerce and communication, and are protected computers under 18 U.S.C. 1030(e)(2).

On information and belief, Russia, the GRU, and GRU Operative #1 knowingly and intentionally accessed the DNC's computers without authorization or in excess of authorization, and thereby obtained and used valuable information from those computers irn violation of 18 US.C. § 1030(a)(2)(C). Such information included, but was not limited to: private, politically sensitive communications between the DNC and Democratic stakeholders and candidates, confidential donor data; confidential campaign strategy plans; opposition and policy research; and documents regarding planned political events, including fundraisers and rallies. The information was used to advance the plan to denigrate the Democrat